In [1]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import math
import gradio as gr 
import numpy as np

# --- Initialize MediaPipe Hands (Global) ---
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7)

# --- Define fingertip landmark IDs (Global) ---
FINGERTIP_IDS = [
    mp_hands.HandLandmark.THUMB_TIP,
    mp_hands.HandLandmark.INDEX_FINGER_TIP,
    mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
    mp_hands.HandLandmark.RING_FINGER_TIP,
    mp_hands.HandLandmark.PINKY_TIP
]

# --- Define labels for the sequential distances (Global) ---
DISTANCE_LABELS = [
    "Thumb-Index",
    "Index-Middle",
    "Middle-Ring",
    "Ring-Pinky"
]

# --- Define a pinch threshold (Global) ---
PINCH_THRESHOLD = 30 

# --- Helper function to calculate Euclidean distance (Global) ---
def calculate_distance(p1, p2):
    """Calculates Euclidean distance between two points (x, y)."""
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# --- THIS IS THE MAIN GRADIO FUNCTION ---
def process_frame(image_from_webcam):
    
    # 1. DO NOT FLIP. Use the raw RGB frame from Gradio
    frame_rgb = image_from_webcam # <-- CHANGE: Removed cv2.flip()

    # 2. Create the BGR version for OpenCV drawing
    frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)

    # 3. Get frame dimensions
    h, w, _ = frame.shape

    # 4. Process the RGB frame
    results = hands.process(frame_rgb)

    # --- Dictionary to store distances for this frame ---
    all_hand_distances = {}

    # 5. Draw landmarks if detected
    if results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(results.multi_hand_landmarks,
                                              results.multi_handedness):

            # Draw hand landmarks ON THE BGR FRAME
            mp_drawing.draw_landmarks(
                frame, 
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style()
            )

            # Extract label (Left/Right) and score
            label = handedness.classification[0].label
            score = handedness.classification[0].score

            # Get wrist coordinates
            wrist = hand_landmarks.landmark[0]
            cx, cy = int(wrist.x * w), int(wrist.y * h)

            # Display label and score near the wrist
            text = f"{label} ({score:.2f})"
            cv2.putText(frame, text, (cx - 20, cy - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)

            # --- Calculate distances & Detect Pinch ---
            
            # Get pixel coordinates for all fingertips
            tip_coords = []
            for tip_id in FINGERTIP_IDS:
                lm = hand_landmarks.landmark[tip_id]
                tip_coords.append((int(lm.x * w), int(lm.y * h)))

            # Calculate distances between sequential fingertips
            sequential_distances = []
            if len(tip_coords) == len(FINGERTIP_IDS): 
                
                # --- 1. Calculate ALL distances for the info box ---
                for i in range(len(tip_coords) - 1):
                    p1 = tip_coords[i]
                    p2 = tip_coords[i+1]
                    dist = calculate_distance(p1, p2)
                    sequential_distances.append(dist)

                # --- 2. Handle ONLY Thumb-Index line and pinch detection ---
                p_thumb = tip_coords[0] # Thumb Tip
                p_index = tip_coords[1] # Index Tip
                pinch_dist = sequential_distances[0]
                
                line_color = (255, 0, 255) # Magenta

                # Check for pinch
                if pinch_dist < PINCH_THRESHOLD:
                    line_color = (0, 255, 0) # Bright Green
                    
                    mid_x = (p_thumb[0] + p_index[0]) // 2
                    mid_y = (p_thumb[1] + p_index[1]) // 2
                    
                    cv2.putText(frame, "PINCH", (mid_x - 30, mid_y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, line_color, 2)
                                
                    cv2.circle(frame, p_thumb, 8, line_color, -1) 
                    cv2.circle(frame, p_index, 8, line_color, -1) 

                cv2.line(frame, p_thumb, p_index, line_color, 2)
            
            all_hand_distances[label] = sequential_distances

    # --- Draw the information box at the bottom ---
    box_height = 130 
    box_y_start = h - box_height
    
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, box_y_start), (w, h), (20, 20, 20), -1)
    
    alpha = 0.6 
    cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0, frame)

    left_col_x = 20
    right_col_x = w // 2 + 20
    text_y_start = box_y_start + 30
    text_y_step = 25 

    # Display Left Hand distances
    if "Left" in all_hand_distances and all_hand_distances["Left"]:
        cv2.putText(frame, "Left Hand", (left_col_x, text_y_start),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        distances = all_hand_distances["Left"]
        for i, (label, dist) in enumerate(zip(DISTANCE_LABELS, distances)):
            text = f"{label}: {dist:.0f}"
            cv2.putText(frame, text, (left_col_x, text_y_start + (i+1) * text_y_step), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Display Right Hand distances
    if "Right" in all_hand_distances and all_hand_distances["Right"]:
        cv2.putText(frame, "Right Hand", (right_col_x, text_y_start),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
        
        distances = all_hand_distances["Right"]
        for i, (label, dist) in enumerate(zip(DISTANCE_LABELS, distances)):
            text = f"{label}: {dist:.0f}"
            cv2.putText(frame, text, (right_col_x, text_y_start + (i+1) * text_y_step), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # 6. Return the processed BGR frame, CONVERTED BACK TO RGB
    return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

# --- Create the Gradio Interface ---
iface = gr.Interface(
    fn=process_frame,
    inputs=gr.Image(sources=["webcam"], streaming=True),
    outputs=gr.Image(),
    live=True,
    title="Live Hand Tracking & Pinch Detection",
    description="Show your hand to the webcam to see live landmark tracking, pinch detection, and finger distances."
)

# --- Launch the app ---
iface.launch(debug=True)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
